In [1]:
import os
os.path.exists('models') or os.chdir("../")
print(os.getcwd())

/home/nitin/github/3D-ResNets-PyTorch


In [12]:
!tree ~/github/kinetics/videos/ --dirsfirst --filelimit=5

/home/nitin/github/kinetics/videos/
├── predict
│   ├── 07xOT83TIG4_000040_000050.mp4
│   ├── 0S-P4lr_c7s_000022_000032.mp4
│   ├── 5gVK5JsNRSc_000005_000015.mp4
│   ├── 5NLQMrXzCQA_000003_000013.mp4
│   └── 6ofkyLo6dns_000102_000112.mp4
├── train
│   ├── archery [10 entries exceeds filelimit, not opening dir]
│   ├── bowling [10 entries exceeds filelimit, not opening dir]
│   ├── flying_kite [10 entries exceeds filelimit, not opening dir]
│   ├── high_jump [10 entries exceeds filelimit, not opening dir]
│   └── marching [10 entries exceeds filelimit, not opening dir]
└── val
    ├── archery [10 entries exceeds filelimit, not opening dir]
    ├── bowling [10 entries exceeds filelimit, not opening dir]
    ├── flying_kite [10 entries exceeds filelimit, not opening dir]
    ├── high_jump [10 entries exceeds filelimit, not opening dir]
    └── marching [10 entries exceeds filelimit, not opening dir]

13 directories, 5 files


In [ ]:
!python ~/github/extract_frames.py \
    -i ~/github/kinetics/videos \
    -o ~/github/kinetics/images \
    -n image \
    -f 10 \
    -s 240x240 \
    -w 8 \
    -r

In [15]:
!tree ~/github/kinetics/images/ --dirsfirst --filelimit=5

/home/nitin/github/kinetics/images/
├── predict
│   ├── 07xOT83TIG4_000040_000050 [100 entries exceeds filelimit, not opening dir]
│   ├── 0S-P4lr_c7s_000022_000032 [100 entries exceeds filelimit, not opening dir]
│   ├── 5gVK5JsNRSc_000005_000015 [74 entries exceeds filelimit, not opening dir]
│   ├── 5NLQMrXzCQA_000003_000013 [73 entries exceeds filelimit, not opening dir]
│   └── 6ofkyLo6dns_000102_000112 [99 entries exceeds filelimit, not opening dir]
├── train
│   ├── archery [10 entries exceeds filelimit, not opening dir]
│   ├── bowling [10 entries exceeds filelimit, not opening dir]
│   ├── flying_kite [10 entries exceeds filelimit, not opening dir]
│   ├── high_jump [10 entries exceeds filelimit, not opening dir]
│   └── marching [10 entries exceeds filelimit, not opening dir]
└── val
    ├── archery [10 entries exceeds filelimit, not opening dir]
    ├── bowling [10 entries exceeds filelimit, not opening dir]
    ├── flying_kite [10 entries exceeds filelimit, not opening dir]

In [9]:
import json
import os
import glob
from os.path import join, basename

class KineticsJSON:
    def get_data(self, path, subset=None):
        data = {}
        subset = subset or path.split("/")[-1]
        files = glob.glob(path +"/*/*")
        for f in files:
            if os.path.isdir(f):
                label = f.split("/")[-2]
                data[basename(f)] = {
                    "subset": subset,
                    "video_path": os.path.realpath(f),
                    "annotations": {"label": label, "segment": (1, len(os.listdir(f)))}
                }
        return data

    def run(self, dataset_path):
        train = join(dataset_path, "train")
        val = join(dataset_path, "val")
        
        data = {}
        data["labels"] = os.listdir(train)
        data["database"] = {}
        data["database"].update(self.get_data(train, "train"))
        data["database"].update(self.get_data(val, "val"))
        return data

data = KineticsJSON().run("/home/nitin/github/kinetics/images")
with open("/home/nitin/github/kinetics/images/kinetics.json", "w") as f:
    f.write(json.dumps(data, indent=4))